# Gold層テーブル作成①: 月別売上分析テーブル

このノートブックでは、Silver層から月別・店舗別・カテゴリ別の売上分析用のGold層テーブルを作成します。

## 目的
- **ビジネス分析用データの提供**: 集約されたKPIデータ
- **パフォーマンス最適化**: 事前集計による高速クエリ
- **トレンド分析**: 時系列での売上動向把握

## 作成テーブル
- **monthly_sales_by_store_and_category**: 月別店舗カテゴリ売上サマリー

## 集計項目
- **monthly_sales**: 月間売上合計
- **monthly_quantity**: 月間販売数量
- **avg_unit_price**: 平均単価

## Gold層の特徴
- ビジネス用途に特化した集約データ
- 高速なダッシュボード表示を実現
- KPIやメトリクス算出の基盤


## 環境設定


In [0]:
# Gold層用の環境設定
catalog_name = "users"              # Unity Catalogのカタログ名
schema = "yukiteru_koide"           # スキーマ名

# カタログとスキーマの切り替え
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema}")

print(f"Gold層処理開始: {catalog_name}.{schema}")

## 月別売上分析テーブルの作成

Silver層のエンリッチされたデータから、月別・店舗別・カテゴリ別の売上統計を集計します。

### 集計軸
- **時間軸**: 年、月
- **場所軸**: 店舗ID、店舗名
- **商品軸**: カテゴリ


In [0]:
# Step 1: Silver層テーブルの読み込み
print("📊 Silver層テーブルを読み込み中...")
df = spark.table(f"{catalog_name}.{schema}.transactions_enriched")
print(f"✅ transactions_enriched テーブル読み込み完了: {df.count()} レコード")

# Step 2: 月別売上集計の実行
print("\n📈 月別売上集計を実行中...")
from pyspark.sql.functions import year, month, sum as _sum, avg

monthly_sales = (
    df
    .withColumn("year", year("transaction_date"))           # 年を抽出
    .withColumn("month", month("transaction_date"))         # 月を抽出
    .groupBy("year", "month", "store_id", "store_name", "category")  # 集計軸でグループ化
    .agg(
        _sum("total_price").alias("monthly_sales"),          # 月間売上合計
        _sum("quantity").alias("monthly_quantity"),          # 月間販売数量
        avg("price").alias("avg_unit_price")                 # 平均単価
    )
)

print(f"🔍 集計結果: {monthly_sales.count()} レコード")

# Step 3: Gold層テーブルとして保存
print("\n💾 Gold層テーブル保存中...")
monthly_sales.write.format("delta").mode("overwrite").saveAsTable("monthly_sales_by_store_and_category")

print("✅ monthly_sales_by_store_and_category テーブル作成完了！")

# 結果のサンプル表示
print("\n📋 作成されたテーブルのサンプル:")
monthly_sales.orderBy("year", "month", "store_id").show(10)

print(f"\n📊 Gold層テーブル統計:")
print(f"  - 総レコード数: {monthly_sales.count()}")
print(f"  - ユニーク店舗数: {monthly_sales.select('store_id').distinct().count()}")
print(f"  - ユニークカテゴリ数: {monthly_sales.select('category').distinct().count()}")
